# 数组

## 1. 删除排序数组中的重复项 (26)

In [3]:
from typing import List
class Solution:
    def removeDuplicates(self, nums: List[int]) -> int:
        slow = fast = 1
        while fast < len(nums):
            if nums[fast-1] != nums[fast]:
                nums[slow] = nums[fast]
                slow += 1
            fast += 1
        return slow

双指针很容易搞定,试试其他操作

In [6]:
from typing import List
class Solution:
    def removeDuplicates(self, nums: List[int]) -> int:
        s = list(set(nums))
        s.sort()
        ans = len(s)
        for i in range(ans):
            nums[i] = s[i]
        return ans

差不多都是o(n)的时间复杂度，第二种方法的set和list操作不好判断时间代价。空间复杂度双指针是o(1)，第二种最坏应该是nums全不重复，那么就是o(n)的复杂度了。

## 2. 买卖股票的最佳时机 Ⅱ

### 重要的dp

感觉很像回溯之类的算法但是可能比我想的简单，或许是单调栈？

In [43]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        n = len(prices)
        stk = [float("Inf")]
        profit = 0
        for i, num in enumerate(prices):
            if num < stk[-1]:
                if len(stk) > 1: profit += stk[-1] - stk[0]
                stk = [num]
            else: stk.append(num)
        if len(stk) > 1: profit += stk[-1] - stk[0]
        return profit

In [44]:
prices = [7,6,4,3,1]

In [45]:
s = Solution()
s.maxProfit(prices)

0

复杂度： 时间和空间应该都是o(n)

最后题解果然是dp，但是单调栈的思路也和贪心有点像了。

考虑到「不能同时参与多笔交易」，因此每天交易结束后只可能存在手里有一支股票或者没有股票的状态。

定义状态 $dp[i][0]$ 表示第i天交易完后手里没有股票的最大利润， $dp[i][1]$ 表示第i天交易完后手里持有一支股票的最大利润（i从0开始）。

考虑 $dp[i][0]$ 的转移方程，如果这一天交易完成后手里没有股票，那木可能的转移状态为前一天已经没有股票，即 $dp[i-1][0]$ ，或者前一天结束的时候手里持有一支股票，即 $dp[i-1][1]$ ，这时候我们要将其卖出，并获得 $prices[i]$ 的收益。因此为了收益最大化，我们列出如下的转移方程：

$$dp[i][0] = max\{dp[i-1][0], dp[i-1][1] + prices[i]\}$$

再来考虑 $dp[i][1]$ ，按照同样的方式考虑转移状态，那么可能的转移状态为前一天已经持有一支股票，即 $dp[i-1][1]$ ，或者前一天结束时还没有股票，即 $dp[i-1][0]$，这时候我们要将其买入，并减少 $prices[i]$ 的收益，可以列出如下的转移方程：

$$dp[i][1] = max\{ dp[i-1][1], dp[i-1][0] - prices[i] \}$$

对于初始状态，根据状态定义我们可以知道第0天交易结束的时候 $dp[0][0] = 0, dp[0][1] = -prices[0]$ 。

因此，我们只要从前往后依次计算状态即可。由于全部交易结束后，持有股票的收益一定低于不持有股票的收益，因此这时候 $dp[n-1][0]$ 的收益必然是大于 $dp[n-1][1]$ 的，最后的答案即为 $dp[n-1][0]$ 。

In [56]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        dp = [0, -prices[0]]
        print(dp)
        for i in range(1,len(prices)):
            new0 = max(dp[0], dp[1] + prices[i])
            new1 = max(dp[1], dp[0] - prices[i])
            dp = [new0, new1]
            print(dp)
        return dp[0]

In [57]:
prices = [1,2,3,4,5]

In [58]:
s = Solution()
s.maxProfit(prices)

[0, -1]
[1, -1]
[2, -1]
[3, -1]
[4, -1]


4

dp的思路真的强，尤其是简化了dp状态的存储以后。只需要用o(1)的空间。感觉对于dp而言最重要的就是状态转移方程，推导出以后剩下的都很easy

## 3. 旋转数组

### 先整体翻转再部分翻转

In [103]:
class Solution:
    def rotate(self, nums: List[int], k: int) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        def reverse(left, right):
            l = right - left
            for i in range(l//2 + 1):
                nums[left + i], nums[right-i] = nums[right - i], nums[left + i]
            # return nums
        n = len(nums)
        reverse(0, n-1)
        reverse(0, k-1)
        reverse(k, n-1)
        return nums

In [104]:
nums = [1,2,3,4,5,6,7]
k = 3

In [105]:
s = Solution()
s.rotate(nums, k)

[5, 6, 7, 1, 2, 3, 4]

## 4. 存在重复元素

In [ ]:
from collections import defaultdict
class Solution:
    def containsDuplicate(self, nums: List[int]) -> bool:
        cnt = defaultdict(int)
        for n in nums:
            cnt[n] += 1
            if cnt[n] == 2: return True
        return False

o(n), o(n)

In [106]:
class Solution:
    def containsDuplicate(self, nums: List[int]) -> bool:
        return len(set(nums)) != len(nums)

o(n), o(n)

排序后再逐个比较的方法时间复杂度为排序复杂度： o(nlogn)，空间为 o(logn)?

## 5. 只出现一次的数字

### 异或

In [109]:
class Solution:
    def singleNumber(self, nums: List[int]) -> int:
        if len(nums) == 1: return nums[0]
        nums.sort()
        for i in range(1, len(nums)-1):
            if nums[i] != nums[i-1] and nums[i] != nums[i+1]: return nums[i]
        if nums[0] != nums[1]: return nums[0]
        if nums[-1] != nums[-2]: return nums[-1]

o(nlogn), o(1)

还是得异或，不过是一块异或而不是两个两个异或

In [121]:
class Solution:
    def singleNumber(self, nums: List[int]) -> int:
        ans = 0
        for n in nums: ans ^= n
        return ans

In [122]:
nums = [1,2,2,3,4,3,4]

In [123]:
s = Solution()
s.singleNumber(nums)

1

### reduce()

reduce() 函数会对参数序列中元素进行累积。

reduce(function, iterable[, initializer])

- function -- 函数，有两个参数
- iterable -- 可迭代对象
- initializer -- 可选，初始参数

即对于可迭代对象的第一第二个元素进行func操作，得到的结果再与第三个元素进行func操作。

In [125]:
class Solution:
    def singleNumber(self, nums: List[int]) -> int:
        return reduce(lambda x, y: x ^ y, nums)

## 6. 两个数组的交集 Ⅱ

In [ ]:
from collections import Counter
class Solution:
    def intersect(self, nums1: List[int], nums2: List[int]) -> List[int]:
        cnt1 = Counter(nums1)
        cnt2 = Counter(nums2)
        ans = []
        for key in cnt1:
            if key in cnt2:
                l = min(cnt1[key], cnt2[key])
                ans.extend([key] * l)
        return ans

哈希表：
- 时间复杂度：o(m+n)
- 空间复杂度：o(m+n)

排序+双指针：
- 时间复杂度：o(mlogm + nlogn)
- 空间复杂度：o(min(m, n))

## 7. 加一

字符串操作

In [2]:
from typing import List
class Solution:
    def plusOne(self, digits: List[int]) -> List[int]:
        num = ""
        for n in digits:
            num += n
        num = str(int(num) + 1)
        ans = []
        for ch in num:
            ans.append(int(ch))
        return ans

模拟

In [6]:
class Solution:
    def plusOne(self, digits: List[int]) -> List[int]:
        carry = 1
        for i in range(len(digits)-1, -1, -1):
             carry, digits[i] = divmod(digits[i] + carry, 10)
        return digits if not carry else [1] + digits

## 8. 移动零

In [40]:
class Solution:
    def moveZeroes(self, nums: List[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        leng = len(nums)
        ptr = 0
        cur = 0
        while ptr < leng:
            while nums[ptr] == 0: ptr += 1
            nums[cur] = nums[ptr]
            cur += 1
            ptr += 1
        for i in range(cur+1, leng): nums[i] = 0

无法处理结尾0的情况

In [54]:
class Solution:
    def moveZeroes(self, nums: List[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        leng = len(nums)
        ptr = 0
        for i, n in enumerate(nums):
            if n == 0: continue
            nums[ptr] = n
            ptr += 1
        for i in range(ptr, leng): nums[i] = 0
        return nums

In [59]:
nums = [0]

In [60]:
s = Solution()
s.moveZeroes(nums)

[0]

o(n), o(1)

## 10. 有效的数独

暴力

In [80]:
from collections import defaultdict
class Solution:
    def isValidSudoku(self, board: List[List[str]]) -> bool:
        cnt = defaultdict(list)
        for i in range(9):
            for j in range(9):
                cur = board[i][j]
                x, y = i // 3, j //3
                if cur.isdigit():
                    if cur in cnt[(i, -1)] or cur in cnt[(-1, j)]\
                    or cur in cnt[(x, y)]: return False
                    cnt[(i, -1)].append(cur)
                    cnt[(-1, j)].append(cur)
                    cnt[(x, y)].append(cur)
                # print(cnt)
        return True

In [83]:
board = \
[[".",".",".",".","5",".",".","1","."],
 [".","4",".","3",".",".",".",".","."],
 [".",".",".",".",".","3",".",".","1"],
 ["8",".",".",".",".",".",".","2","."],
 [".",".","2",".","7",".",".",".","."],
 [".","1","5",".",".",".",".",".","."],
 [".",".",".",".",".","2",".",".","."],
 [".","2",".","9",".",".",".",".","."],
 [".",".","4",".",".",".",".",".","."]]

In [82]:
s = Solution()
s.isValidSudoku(board)

False

时间：o(n^2) 空间：o(n^2)

限制在9*9数独的话则都是常数复杂度

## 11. 旋转图像

不会原地

顺时针 90° = 水平翻转 + 转置

In [113]:
class Solution:
    def rotate(self, matrix: List[List[int]]) -> None:
        """
        Do not return anything, modify matrix in-place instead.
        """
        n = len(matrix)
        for i in range(n // 2):
            matrix[i], matrix[n - i - 1] = matrix[n - i - 1], matrix[i]
        for i in range(n):
            for j in range(i):
                print(matrix[i][j], matrix[j][i])
                matrix[i][j], matrix[j][i] = matrix[j][i], matrix[i][j]
        return matrix

In [114]:
matrix = [[5,1,9,11],[2,4,8,10],[13,3,6,7],[15,14,12,16]]

In [115]:
s = Solution()
s.rotate(matrix)

13 14
2 12
4 6
5 16
1 7
9 10


[[15, 13, 2, 5], [14, 3, 4, 1], [12, 6, 8, 9], [16, 7, 10, 11]]

学费了

时间: o(n^2), 空间: o(1)

## 小结：

- 对于数组问题感觉双指针与哈希表比较常用；
- 很多问题在对数组进行排序后都会变得没那么复杂；
- 异或运算对于寻找只出现一次的数字绝杀；
- 要记得一种可以带着数组下标进行排序的思路（870 优势洗牌）；

# 字符串

## 1. 反转字符串

In [117]:
class Solution:
    def reverseString(self, s: List[str]) -> None:
        """
        Do not return anything, modify s in-place instead.
        """
        n = len(s)
        for i in range(n // 2):
            s[i], s[n - i - 1] = s[n - i - 1], s[i]
        return s

In [120]:
s = ["H","a","n","n","a","h"]

In [121]:
S = Solution()
S.reverseString(s)

['h', 'a', 'n', 'n', 'a', 'H']

## 2. 整数反转

In [125]:
class Solution:
    def reverse(self, x: int) -> int:
        ans = int(str(abs(x))[::-1])
        if ans > 2**31 - 1: return 0
        return -ans if x < 0 else ans

In [126]:
x = 210

In [127]:
s = Solution()
s.reverse(x)

12

o(n), o(1)

## 3. 字符串中的第一个唯一字符

In [128]:
# 先逃课
from collections import Counter
class Solution:
    def firstUniqChar(self, s: str) -> int:
        cnt = Counter(s)
        for i, ch in enumerate(s):
            if cnt[ch] == 1: return i
        return -1

In [133]:
s = "leetcode"

In [134]:
S = Solution()
S.firstUniqChar(s)

0

o(n), o($\Sigma$) 其中Sigma是字符集

In [142]:
from collections import defaultdict
class Solution:
    def firstUniqChar(self, s: str) -> int:
        cnt = defaultdict(int)
        slow = 0
        for i in range(len(s)):
            cnt[s[i]] += 1
            while cnt[s[slow]] >= 2: 
                slow += 1
                if slow == len(s): return -1
        return slow

In [145]:
s = "aabbccdeeff"

In [146]:
S = Solution()
S.firstUniqChar(s)

6

时间貌似退化为了0(n^2)

### 队列：
- 我们也可以借助队列找到第一个不重复的字符。队列具有「先进先出」的性质，因此很适合用来找出第一个满足某个条件的元素。

In [154]:
from collections import deque
class Solution:
    def firstUniqChar(self, s: str) -> int:
        position = dict()
        q = deque()
        n = len(s)
        for i, ch in enumerate(s):
            print(position, q)
            if ch not in position:
                position[ch] = i
                q.append((s[i], i))
            else:
                position[ch] = -1
                while q and position[q[0][0]] == -1:
                    q.popleft()
        return -1 if not q else q[0][1]

In [157]:
s = "loveleetcode"

In [158]:
S = Solution()
S.firstUniqChar(s)

{} deque([])
{'l': 0} deque([('l', 0)])
{'l': 0, 'o': 1} deque([('l', 0), ('o', 1)])
{'l': 0, 'o': 1, 'v': 2} deque([('l', 0), ('o', 1), ('v', 2)])
{'l': 0, 'o': 1, 'v': 2, 'e': 3} deque([('l', 0), ('o', 1), ('v', 2), ('e', 3)])
{'l': -1, 'o': 1, 'v': 2, 'e': 3} deque([('o', 1), ('v', 2), ('e', 3)])
{'l': -1, 'o': 1, 'v': 2, 'e': -1} deque([('o', 1), ('v', 2), ('e', 3)])
{'l': -1, 'o': 1, 'v': 2, 'e': -1} deque([('o', 1), ('v', 2), ('e', 3)])
{'l': -1, 'o': 1, 'v': 2, 'e': -1, 't': 7} deque([('o', 1), ('v', 2), ('e', 3), ('t', 7)])
{'l': -1, 'o': 1, 'v': 2, 'e': -1, 't': 7, 'c': 8} deque([('o', 1), ('v', 2), ('e', 3), ('t', 7), ('c', 8)])
{'l': -1, 'o': -1, 'v': 2, 'e': -1, 't': 7, 'c': 8} deque([('v', 2), ('e', 3), ('t', 7), ('c', 8)])
{'l': -1, 'o': -1, 'v': 2, 'e': -1, 't': 7, 'c': 8, 'd': 10} deque([('v', 2), ('e', 3), ('t', 7), ('c', 8), ('d', 10)])


2